In [29]:
import numpy as np
import cv2

# creating a haar-cascade object for face detection
face_cas = cv2.CascadeClassifier('./haarcascade_frontalface_alt2.xml')

# instantiate a camera object to capture images
video = cv2.VideoCapture(0)

font = cv2.FONT_ITALIC

# loading the data
f_1 = np.load('./face_01.npy').reshape((20 , 50*50*3))
f_2 = np.load('./face_02.npy').reshape((20 , 50*50*3))
f_3 = np.load('./face_03.npy').reshape((20 , 50*50*3))
# printing the shape
print(f_1.shape , f_2.shape , f_3.shape)

# creating a look up table for names
names = {
    0 : 'Sarthak',
    1 : 'Sarthakpics',
    2 : 'notSarthak'
}

# storing the label in a matrix
labels = np.zeros((60,1))
labels[: 20 , :] = 0.0
labels[20 : 40 , :] = 1.0
labels[ 40 : , : ] = 2.0

# concatenating all data
data = np.concatenate([f_1 , f_2])
print(data.shape , labels.shape)

# defining euclidean distance
def dist(x1 , x2):
    return np.sqrt(((x1-x2)**2).sum())

# defining knn algorithm
def knn(query_point , train , labels , k= 17):
    vals = []
    for idx in range(train.shape[0]):
        v = [dist(query_point , train[idx , :]) , labels[idx]]
        vals.append(v)
    
    updated_vals = sorted(vals, key = lambda x : x[0])
    
    pred_ar = np.array(updated_vals[:k])
    
    pred_ar = np.unique(pred_ar[: , 1] , return_counts=True)
    
    pred_idx = pred_ar[1].argmax()
    
    return pred_ar[0][pred_idx]


while True:
    check , frame = video.read()
    if(check == False):
        break
    # converting the current frame to grayscale
    gray = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY)
    # Here in order to detect faces in the current frame , we'll use Haar Cascade which is a 
    # classifier to detect faces in the image frame. Also , 1.3 and 5 are fine tuning
    #parameters
    faces = face_cas.detectMultiScale(gray , 1.3 , 5)
    
    for x , y , w , h in faces:
        # obtaining the face component from the image frame
        component_of_face = frame[y:y+h , x:x+w , :]
        # resize the face image to 50X50X3
        # NOTE: Resizing is necessary so that recognisor can work on equally sized images
        fc = cv2.resize(component_of_face , (50 , 50))
        
        label = knn(fc.flatten() , data , labels)
        
        # get the name associated to label generated by knn algorithm
        name = names[int(label)]
        
        # print the name around the frame 
        cv2.putText(frame , name , (x , y) , font , 1 , (255 , 255 , 0) , 2)
        
        # draw rectangle around the face of the person in front of camera
        cv2.rectangle(frame , (x , y) , (x+w , y+h) , (255 , 255 , 0 ) , 2)
    
    cv2.imshow('Hello' , frame)
    
    key = cv2.waitKey(1)
    if(key == ord('q')):
        break
    

# now we destroy the windows we have created
video.release()
cv2.destroyAllWindows()

(20, 7500) (20, 7500) (20, 7500)
(40, 7500) (60, 1)
